In [1]:
%matplotlib inline
import pandas as pd
import os

In [31]:
fpath = '/media/sf_VBox_Shared/networks/trade/'

To do: read the multiple year file, but do smart filtering so that we do not load to much into memory

In [2]:
df = pd.read_csv('/media/sf_VBox_Shared/networks/trade/OECD-WTO_BATIS_data.csv', 
                 keep_default_na=False,
                 na_values=[''],
                dtype={'Reporter':str, 'Partner': str, 'Flow': str, 
                       'Year': int,
                       'Reported_value': float,
                       'Final_value': float,
                       'Final_value_methodology': str,
                       'Balanced_value': float})

In [50]:
#df = pd.read_csv(os.path.join(fpath, 'services_annual_dataset.csv'), keep_default_na=False)

In [8]:
df = df[~df.Reported_value.isnull()]
df = df[~df.Balanced_value.isnull()]

In [13]:
df.head()

,Reporter,Partner,Flow,Item_code,Year,Reported_value,Final_value,Final_value_methodology,Balanced_value
2513,888,WL,M,S200,2008,396.19,396.19,NaN,1870.3467
2707,888,WL,M,S200,2009,399.67,399.67,E2,1736.8767
2900,888,WL,M,S200,2010,510.13,510.13,E2,1694.2854
3092,888,WL,M,S200,2011,511.84,511.84,E2,1731.8951
3285,888,WL,M,S200,2012,374.05,374.05,E2,1738.5639


In [19]:
df.Item_code.value_counts()

S200    120998
S205     57395
S236     57043
S268     46914
S260     46116
S253     44799
S291     43851
S245     43044
S266     42070
S262     41717
S249     39667
S287     38633
Name: Item_code, dtype: int64

Create one 'import' table 

In [14]:
countries = pd.read_csv('/media/sf_VBox_Shared/CorpNet/board-interlock/countries.csv',
                        keep_default_na=False, na_values=[''])
countries.head()

,id,Label,latitude,longitude,name
0,AD,Andorra,42.546245,1.601554,Andorra
1,AE,United Arab Emirates,23.424076,53.847818,United Arab Emirates
2,AF,Afghanistan,33.939110,67.709953,Afghanistan
3,AG,Antigua and Barbuda,17.060816,-61.796428,Antigua and Barbuda
4,AI,Anguilla,18.220554,-63.068615,Anguilla


In [15]:
country_ids = set(countries.id)

In [16]:
len(country_ids), df.Reporter.nunique()

(248, 191)

In [22]:
df_import = df[(df.Flow=='M') & (df.Reporter.isin(country_ids)) & (df.Partner.isin(country_ids))]
df_import = df_import[df_import.Item_code=='S200']

In [26]:
df_import.Year.unique()

array([1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005,
       2006, 2007, 2008, 2009, 2010, 2011, 2012])

In [23]:
df_group_import =  df_import.groupby(
    ['Year', 'Reporter', 'Partner'])['Balanced_value'].sum()
df_group_import.shape

(56225,)

In [24]:
# Filter less than 10000 dollar
df_group_import = df_group_import[df_group_import>0.01]
df_group_import.shape

(52884,)

In [27]:
df_group_import.name = 'Weight'
df_group_import = df_group_import.reset_index()
df_group_import = df_group_import.rename(columns={'Reporter': 'Target', 'Partner': 'Source'})

In [28]:
df_group_import.head()

,Year,Target,Source,Weight
0,1995,AT,AL,1.3956
1,1995,AT,AR,13.1798
2,1995,AT,AU,60.6951
3,1995,AT,BE,319.7563
4,1995,AT,BG,33.3532


In [32]:
df_group_import.to_csv(os.path.join(fpath, 'import', 'import-links-allyears.csv'), header=True, index=False)

In [34]:
for year in df_import.Year.unique():
    df_year = df_group_import[df_group_import.Year==year][['Source', 'Target', 'Weight']]
    df_year.to_csv(os.path.join(fpath,'import', 'import-links-{}.csv'.format(year)), header=True, index=False)

In [116]:
import networkx as nx
countries = countries.set_index('id')

In [137]:
countries = countries.fillna(0)

In [138]:
nw = nx.from_pandas_edgelist(df_group_import, source='Source', target='Target', edge_attr='Weight', 
                             create_using=nx.DiGraph())

In [141]:
for c in countries.columns:
    values = dict(countries[c])
    if countries[c].dtype == 'float64':
        print(c)
        values = {k: float(values[k]) for k in values}
    nx.set_node_attributes(nw, name=c, values=values)

latitude
longitude


In [142]:
nx.write_gexf(nw, '/media/sf_VBox_Shared/networks/trade/import-links.gexf')

In [5]:
df.Flow_Description.value_counts()

Exports    1257045
Imports    1244239
Name: Flow_Description, dtype: int64

In [4]:
df.Indicator_Description.value_counts()

Services                                                                                          99503
Goods-related services                                                                            70382
Transport                                                                                         39976
Travel                                                                                            38768
Other services                                                                                    37771
Commercial services                                                                               37163
Other business services                                                                           35749
Telecommunications, computer, and information services                                            35563
Insurance and pension services                                                                    34626
Other commercial services                                       